In [37]:
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

# import chainlit as cl
# from dotenv import load_dotenv

from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

from llama_index import (
    ServiceContext,
)
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)

from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
from llama_index.node_parser import SentenceSplitter

from llama_index import SimpleDirectoryReader

In [38]:
document_list = [
	'../quant_scraper/docs/vbt_pro/cookbook-persistence.txt',
	'../quant_scraper/docs/vbt_pro/documentation-building-blocks.txt',
]

## DefaultVectorStorage

In [39]:
llm_embed = OpenAIEmbedding(model="text-embedding-3-small")
llm_chat = OpenAI(model="gpt-4-turbo-preview", temperature=0.1)

service_context = ServiceContext.from_defaults(
	llm=llm_chat,
	chunk_size=4096,
	embed_model=llm_embed,
	# callback_manager=CallbackManager([cl.LlamaIndexCallbackHandler()]),
)

In [40]:
print('Loading documents...')
# documents = SimpleDirectoryReader(input_dir='../quant_scraper/docs/', recursive=True).load_data(num_workers=4, show_progress=True)
documents = SimpleDirectoryReader(input_files=document_list).load_data(num_workers=4, show_progress=True)

len(documents)

Loading documents...


2

In [41]:
parser = SentenceSplitter(chunk_size=2048, chunk_overlap=0, separator="\n\n#  ")
nodes = parser.get_nodes_from_documents(documents)

len(nodes)

10

In [42]:
print('Building index...')
index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context)

Building index...


Generating embeddings: 100%|██████████| 5/5 [00:01<00:00,  3.24it/s]


In [43]:
# resposne = index.as_chat_engine().chat('overview vbt pro cookbook')
# print(resposne)

In [44]:
retriever = VectorIndexRetriever(
	index=index,
	similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
	response_mode="tree_summarize", service_context=service_context)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)


In [45]:
# index.as_query_engine().query("How do I use the vbt pro library?")

In [46]:
test_query = """
In my strategy I have 4 take profits and stop loss. How can I move stop loss to breakeven after the first take profit is hit?
"""

response = query_engine.query(test_query)
print(response)

To implement a strategy where the stop loss is moved to breakeven after the first take profit is hit, you can follow a structured approach. This involves tracking the status of your take profits and adjusting the stop loss accordingly. Here's a general outline of how you might achieve this:

1. **Define Your Entry, Take Profit, and Stop Loss Levels**: Initially, you need to define where your entry point is, where your take profits are located, and where your initial stop loss is placed. The take profits can be defined as a percentage or price levels above your entry point, and the stop loss can be a percentage or price level below your entry point.

2. **Track the First Take Profit Hit**: You need a mechanism to track when your first take profit level is hit. This can be done by monitoring the price and comparing it with your predefined take profit levels.

3. **Move Stop Loss to Breakeven**: Once the first take profit is hit, you adjust your stop loss to the breakeven point, which is 

In [47]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='68d0cdf5-4284-42ee-ad0d-44e531f2e967', embedding=None, metadata={'file_path': '../quant_scraper/docs/vbt_pro/cookbook-persistence.txt', 'file_name': 'cookbook-persistence.txt', 'file_type': 'text/plain', 'file_size': 3968, 'creation_date': '2024-01-28', 'last_modified_date': '2024-01-28', 'last_accessed_date': '2024-01-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='76f89382-71ad-4766-9278-5bc0b5fe5a00', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '../quant_scraper/docs/vbt_pro/cookbook-persistence.txt', 'file_name': 'cookbook-persistence.txt', 'file_type': 'text/plain', 'file_size': 3968, 'creation_date': '2024-01-28', 'last_modified_date': '2024-01-

In [48]:
index.set_index_id("default")
index.storage_context.persist(persist_dir="../index_notebook/")

In [49]:
from llama_index import StorageContext, load_index_from_storage
from llama_index.vector_stores import SimpleVectorStore
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore

storage_context2 = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="../index_notebook/"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="../index_notebook/", namespace="default"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="../index_notebook/"),
)

# index2 = VectorStoreIndex.from_vector_store(vector_store2, service_context=service_context)
index2 = load_index_from_storage(storage_context=storage_context2, index_id="default")

In [50]:
retriever2 = VectorIndexRetriever(
	index=index2,
	similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer2 = get_response_synthesizer(
	response_mode="tree_summarize", service_context=service_context)

# assemble query engine
query_engine2 = RetrieverQueryEngine(
    retriever=retriever2,
    response_synthesizer=response_synthesizer2,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)


In [55]:
test_query = """
In my strategy I have 4 take profits and stop loss. How can I move stop loss to breakeven after the first take profit is hit?
"""

response = query_engine2.query(test_query)
print(response)

To move the stop loss to breakeven after the first take profit is hit in your strategy, you can follow these steps:

1. **Define Your Entry Price and Stop Loss**: Initially, when you enter a trade, you define an entry price and a stop loss. The entry price is the price at which you enter the trade, and the stop loss is the price at which you exit the trade if it moves against you to limit your losses.

2. **Set Take Profit Levels**: Since you have 4 take profit levels, you need to define these levels. Each take profit level is a price at which a portion of your trade will be closed in profit. The first take profit level is the one after hitting which you want to move your stop loss to breakeven.

3. **Calculate Breakeven Point**: The breakeven point is your entry price. Moving the stop loss to breakeven means adjusting the stop loss to the entry price so that if the price reverses after hitting the first take profit, you exit the trade without a loss.

4. **Implement Logic to Adjust St

In [53]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='23a68867-577b-45cd-8c81-324775b1b92b', embedding=None, metadata={'file_path': '../quant_scraper/docs/vbt_pro/documentation-building-blocks.txt', 'file_name': 'documentation-building-blocks.txt', 'file_type': 'text/plain', 'file_size': 59262, 'creation_date': '2024-01-28', 'last_modified_date': '2024-01-28', 'last_accessed_date': '2024-01-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a2798d13-a1e0-43f3-81e1-37ac37c69ea9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '../quant_scraper/docs/vbt_pro/documentation-building-blocks.txt', 'file_name': 'documentation-building-blocks.txt', 'file_type': 'text/plain', 'file_size': 59262, 'creation_date': '2024-0

In [56]:
chat_engine = index2.as_chat_engine(chat_mode="react", verbose=True)

In [58]:
response = chat_engine.chat("In my strategy I have 4 take profits and stop loss. How can I move stop loss to breakeven after the first take profit is hit?")
print(response)

Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'How to move stop loss to breakeven after the first take profit is hit in a trading strategy?'}
Observation: To move the stop loss to breakeven after the first take profit is hit in a trading strategy, you can use a conditional statement to check if the take profit condition has been met. If it has, you can then update the stop loss level to the entry price. This will ensure that you lock in profits and protect your position from potential losses.
Thought: I can answer without using any more tools.
Answer: To move the stop loss to breakeven after the first take profit is hit in a trading strategy, you can use a conditional statement to check if the take profit condition has been met. If it has, you can then update the stop loss level to the entry price. This will ensure that you lock in profits and protect your position from potential losses.
To move the stop loss to breakeve

In [60]:
response = chat_engine.chat("Can you now show me the code of the strategy we discussed?")
print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: Certainly! Here's an example code snippet that demonstrates how to move the stop loss to breakeven after the first take profit is hit in a trading strategy:

```python
# Initialize variables
take_profit_levels = [100, 200, 300, 400]  # List of take profit levels
stop_loss = 50  # Stop loss level
entry_price = 150  # Entry price
current_price = 250  # Current price

# Check if the first take profit level is hit
if current_price >= take_profit_levels[0]:
    # Move stop loss to breakeven
    stop_loss = entry_price

# Print the updated stop loss level
print("Updated stop loss:", stop_loss)
```

In this code, we have a list `take_profit_levels` that contains the desired take profit levels. We also have the `stop_loss` level and the `entry_price` of the trade. The `current_price` represents the current market price.

The code checks if the `current_price` is greater than or equal to the first take profit level (`take_profit_l

In [61]:
response = chat_engine.chat("What do you know about vectorbt pro?")
print(response)

Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is Vectorbt Pro?'}
Observation: Vectorbt Pro is a computational library that provides modules for processing and plotting time series and numeric data. It includes an arsenal of Numba-compiled functions for accelerating and extending Pandas. It also offers builder mixins, which are classes that allow specific functionality to be built from a class's attributes. Vectorbt Pro aims to improve performance by using NumPy and Numba for computations, while still maintaining the ability to present results in a Pandas format.
Thought: I can answer without using any more tools.
Answer: Vectorbt Pro is a computational library that provides modules for processing and plotting time series and numeric data. It includes Numba-compiled functions for accelerating and extending Pandas, as well as builder mixins for building specific functionality. Vectorbt Pro aims to improve performance

In [62]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='23a68867-577b-45cd-8c81-324775b1b92b', embedding=None, metadata={'file_path': '../quant_scraper/docs/vbt_pro/documentation-building-blocks.txt', 'file_name': 'documentation-building-blocks.txt', 'file_type': 'text/plain', 'file_size': 59262, 'creation_date': '2024-01-28', 'last_modified_date': '2024-01-28', 'last_accessed_date': '2024-01-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a2798d13-a1e0-43f3-81e1-37ac37c69ea9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '../quant_scraper/docs/vbt_pro/documentation-building-blocks.txt', 'file_name': 'documentation-building-blocks.txt', 'file_type': 'text/plain', 'file_size': 59262, 'creation_date': '2024-0

In [63]:
chat_engine2 = index2.as_chat_engine(chat_mode="best", verbose=True)

In [66]:
response = chat_engine2.chat("In my strategy I have 4 take profits and stop loss. How can I move stop loss to breakeven after the first take profit is hit?")
print(response)

Added user message to memory: In my strategy I have 4 take profits and stop loss. How can I move stop loss to breakeven after the first take profit is hit?
To move the stop loss to breakeven after the first take profit is hit in your strategy with 4 take profits and a stop loss, you can follow these steps:

1. Set your initial stop loss level at a certain distance from your entry price.
2. When the price reaches your first take profit level, adjust your stop loss level to your entry price (breakeven).
3. Continue to monitor the price movement and adjust your remaining take profit levels accordingly.

Here's an example to illustrate this strategy:

Let's say you enter a trade with a stop loss of $100 and four take profit levels: TP1 at $50, TP2 at $100, TP3 at $150, and TP4 at $200.

1. Set your initial stop loss level at $100.
2. When the price reaches TP1 at $50, adjust your stop loss level to your entry price (breakeven), which is $100.
3. If the price continues to move in your favor